In [3]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import weaviate
import weaviate.classes as wvc
import os
import requests
import json
from dotenv import load_dotenv
load_dotenv()
import openai
import dspy
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShot, BootstrapFewShotWithRandomSearch, BootstrapFinetune
from dspy.retrieve.weaviate_rm import WeaviateRM
openai.api_key = os.getenv("OPENAI_API_KEY")
weaviate_key = os.getenv("WEAVIATE_API_KEY")

In [4]:
data_path = '/Users/maiti/Desktop/projects/bookstore-copilot/data/booksummaries.txt'
book_summaries = pd.read_csv(data_path, delimiter='\t', names=['wikipedia_id', 'freebase_id', 'title', 'author', 'publication_date', 'genres', 'summary'])

In [5]:
book_summaries.head()

,wikipedia_id,freebase_id,title,author,publication_date,genres,summary
0,620,/m/0hhy,Animal Farm,George Orwell,1945-08-17,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca..."
1,843,/m/0k36,A Clockwork Orange,Anthony Burgess,1962,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan..."
2,986,/m/0ldx,The Plague,Albert Camus,1947,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...
3,1756,/m/0sww,An Enquiry Concerning Human Understanding,David Hume,NaN,NaN,The argument of the Enquiry proceeds by a ser...
4,2080,/m/0wkt,A Fire Upon the Deep,Vernor Vinge,NaN,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...",The novel posits that space around the Milky ...


In [6]:
# Connecting to weaviate
WCS_CLUSTER_URL = "https://bookstore-copilot-w6cbjbif.weaviate.network"

client = weaviate.connect_to_wcs(
    cluster_url=WCS_CLUSTER_URL,
    auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WEAVIATE_API_KEY")),
    headers={
        "X-OpenAI-Api-Key": openai.api_key  # Replace with your inference API key
    },
    skip_init_checks=True,
)

## Creating Collection In Weaviate Using Python
This is the method to create a collection in weaviate. I have not put it in a cell since, because it needs to be created only once. 
```python
client.collections.create(
        "book_summaries",
        vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(),
        vector_index_config=wvc.config.Configure.VectorIndex.hnsw(),
        properties=[
            wvc.config.Property(name="book_summaries", data_type=wvc.config.DataType.TEXT),
        ]
)
```

### Adding Bookings to Weaviate with Python
This is the method to create a collection in weaviate. I have not put it in a cell since, because it needs to be created only once. 
```python
book_summary_collection = client.collections.get("book_summaries")
total_rows = 2000
for idx, row in book_summaries.iterrows():
    try:
        title = row['title']
        summary = row['summary']
        genres = " ".join(eval(row['genres']).values()) if row['genres'] else ""
        author = row['author']
        uuid = book_summary_collection.data.insert(
            {
                "book_summaries": summary,
                "title": title,
                "genres": genres,
                "author": author,
            }
        )
        print(f"Inserted {title} with UUID {uuid}")
    except Exception as e:
        print(f"Error inserting {title}: {e}")

    if idx > total_rows:
        break
```

In [9]:

questions = client.collections.get("book_summaries")

response = questions.query.near_text(
    query="Harry Potter",
    limit=2
)

print(response.objects[0].properties)  # Inspect the first object

{'title': 'Harry Potter and the Order of the Phoenix', 'book_summaries': ' Harry Potter is spending another summer with his dreadful Aunt Petunia and Uncle Vernon. when a pair of Dementors stage an unexpected attack on Harry and his cousin Dudley. After he uses magic to defend himself and Dudley, he is temporarily expelled from Hogwarts for using magic outside of the school, despite being legally allowed to do in self-defence, before it is rescinded. A few days afterwards, Harry is visited by a group of wizards and Mad-Eye Moody and is whisked off to Number 12, Grimmauld Place, London, the home of Harry\'s godfather, Sirius Black, and the headquarters of the Order of the Phoenix. As Harry learns from his best friends, Ron Weasley and Hermione Granger The Order is a group of witches and wizards, led by Hogwarts headmaster Albus Dumbledore, dedicated to fighting the evil Lord Voldemort and his followers. The Order is forced to operate in secrecy, outside of the jurisdiction of the Minist

In [10]:
# setting up DSPy
llm = dspy.OpenAI(model="gpt-3.5-turbo")
dspy.settings.configure(lm=llm)

In [11]:
book_collection = client.collections.get('book_summaries')

In [12]:
prompt = "Write about the book: {title}"


response = book_collection.query.near_text(
    query="World history",
    limit=2
)

In [13]:
response.objects[0].properties['author']

'Robert Silverberg'

In [57]:
class FunctionCalling(dspy.Signature):
    """" There are 5 functions and their function signature.
    1. Book search by title get_books_by_title(query: str) -> List[str]
    2. Book seach by author get_books_by_author(query: str) -> List[str]
    3. Book seach by genre get_books_by_genre(query: str) -> List[str]
    4. Book recommendations based on user preferences get_recommendations(query: str) -> List[str]
    5. More about the book get_more_about_the_book(title: str) -> str
    5. General enquires about shipping, policies and returns get_general_enquires(query: str) -> str
    Based on the user input, the module will call the appropriate function with the required parameters inferred from the query and return the response.
    """
    query = dspy.InputField(desc='Input query')
    function_name = dspy.OutputField(desc='Function name to be called along with the parameters')

class BookInterestFromQuery(dspy.Signature):
    """This module will take the user query and infer a string which has all necessary attributes that the
    user wants to search for.
    For e.g. I want a book on magic and adventure -> infer 'magic adventure' as the user preferences
    """
    query = dspy.InputField(desc='Input query')
    user_preferences = dspy.OutputField(desc='Inferred user preferences')

class GenreFromQuery(dspy.Signature):
    """This module will take the user query and infer the genre of the book that the user is interested in.
    For e.g. I want a book on magic and adventure -> infer 'fantasy' as the genre
    """
    query = dspy.InputField(desc='Input query')
    genre = dspy.OutputField(desc='Inferred genre')

class AuthorFromQuery(dspy.Signature):
    """This module will take the user query and infer the author of the book that the user is interested in.
    For e.g. I want a book by J.K. Rowling -> infer 'J.K. Rowling' as the author
    """
    query = dspy.InputField(desc='Input query')
    author = dspy.OutputField(desc='Inferred author')

class TitleFromQuery(dspy.Signature):
    """This module will take the user query and infer the title of the book that the user is interested in.
    For e.g. I want a book with the title Harry Potter -> infer 'Harry Potter' as the title
    """
    query = dspy.InputField(desc='Input query')
    title = dspy.OutputField(desc='Inferred title')

class GeneralEnquiresFromQuery(dspy.Signature):
    """This module will take the user query and infer the general enquires that the user is interested in.
    There are only 3 categories possible, shipping, policies and returns.
    For e.g. I want to know about shipping -> infer 'shipping' as the general enquires
    """
    query = dspy.InputField(desc='Input query')
    general_enquires = dspy.OutputField(desc='Inferred general enquires')

class EnquiryAnswerGenerator(dspy.Signature):
    """Answer general queries by the customer. General queries can be related to shipping, returns and policies.
    You are Amazon's virtual assistant and you have to answer the general queries of the customer.
    There are only 3 categories possible, shipping, policies and returns.
    """
    query = dspy.InputField(desc='General enquires')
    response = dspy.OutputField(desc='Response to the general enquires')

class MoreAboutTheBook(dspy.Signature):
    """Given the title of the book, the module will return the summary of the book.
    """
    title = dspy.InputField(desc='Title of the book')
    summary = dspy.OutputField(desc='Summary of the book')

book_interest_from_query = dspy.ChainOfThought(BookInterestFromQuery, n=2)
title_from_query = dspy.ChainOfThought(TitleFromQuery, n=2)
genre_from_query = dspy.ChainOfThought(GenreFromQuery, n=2)
author_from_query = dspy.ChainOfThought(AuthorFromQuery, n=2)
general_enquires_from_query = dspy.ChainOfThought(GeneralEnquiresFromQuery, n=2)
enquiry_answer_generator = dspy.ChainOfThought(EnquiryAnswerGenerator, n=2)
function_calling = dspy.ChainOfThought(FunctionCalling, n=2)
more_about_the_book = dspy.ChainOfThought(MoreAboutTheBook, n=2)

def get_books_by_title(query: str) -> str:
    title = book_interest_from_query(query=query).title
    response = book_collection.query.near_text(
        query=title,
        limit=4
    )
    res = [obj.properties['title'] + ' by ' + obj.properties['author'] for obj in response.objects]
    return res

def get_books_by_author(query: str) -> str:
    author = author_from_query(query=query).author
    response = book_collection.query.near_text(
        query=author,
        limit=4
    )
    res = [obj.properties['title'] + ' by ' + obj.properties['author'] for obj in response.objects]
    return res

def get_books_by_genre(query: str) -> str:
    genre = genre_from_query(query=query).genre
    response = book_collection.query.near_text(
        query=genre,
        limit=4
    )
    res = [obj.properties['title'] + ' by ' + obj.properties['author'] for obj in response.objects]
    return res

def get_recommendations(query: str) -> str:
    user_preferences = book_interest_from_query(query=query).user_preferences
    response = book_collection.query.near_text(
        query=user_preferences,
        limit=4
    )
    res = [obj.properties['title'] + ' by ' + obj.properties['author'] for obj in response.objects]
    return res

def get_general_enquires(query: str) -> str:
    general_enquires = enquiry_answer_generator(query=query).response
    return general_enquires

def get_more_about_the_book(title: str) -> str:
    title = title_from_query(query=title).title
    response = book_collection.query.near_text(
        query=title,
        limit=1
    )
    summary = response.objects[0].properties['book_summaries']
    return summary


In [58]:
class ResponseCrafter(dspy.Signature):
    """ You will given some data and a query from the user. You need to craft a polite response, for the query, using the data provided.
    """
    response = dspy.InputField(desc='Response')
    query = dspy.InputField(desc='User query')
    crafted_response = dspy.OutputField(desc='Crafted response')

In [59]:
response_crafter = dspy.ChainOfThought(ResponseCrafter, n=2)
while 1:
    query = input("Enter your query: ")
    if query == 'exit':
        break
    function_name = function_calling(query=query).function_name
    res = eval(function_name)
    if isinstance(res, list):
        res = '\n'.join(res)
    response = response_crafter(response=res, query=query).crafted_response
    print(response)


Our return policy allows customers to return eligible items within 30 days of receipt for a full refund. Items must be in their original packaging and in new, unused condition. Certain items, such as perishable goods or personalized items, may not be eligible for return. Customers can initiate a return through their Amazon account and choose to have a prepaid return label provided or return the item at a drop-off location. Refunds are typically processed within 3-5 business days after the item is received.
